In [1]:
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('ZHIPU_API_KEY')
base_url = "https://open.bigmodel.cn/api/paas/v4/"
chat_model = "glm-4-flash"
emb_model = "embedding-2"

# 配置对话模型
from llama_index.llms.zhipuai import ZhipuAI
llm = ZhipuAI(
    api_key = api_key,
    model = chat_model,
)

# 配置嵌入模型
from llama_index.embeddings.zhipuai import ZhipuAIEmbedding
embedding = ZhipuAIEmbedding(
    api_key = api_key,
    model = emb_model,
)
emb = embedding.get_text_embedding("你好呀呀")

# 从指定文件读取，输入为List
from llama_index.core import SimpleDirectoryReader,Document
documents = SimpleDirectoryReader(input_files=['./docs/问答手册.txt']).load_data()

# 构建向量索引
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_documents(documents,embed_model=embedding)
# 想要看到进度条的话，加一个参数 show_progress=True
# index = VectorStoreIndex.from_documents(documents,embed_model=embedding,show_progress=True)

文档管理意味着对保存在硬盘上的index进行增删改查。我们先来看看针对faiss向量存储的管理方式。

查看index下面的所有文档

In [2]:
print(index.docstore.docs)


{'cdf10de3-c206-4796-b6b2-3720e6f342dd': TextNode(id_='cdf10de3-c206-4796-b6b2-3720e6f342dd', embedding=None, metadata={'file_path': 'docs/问答手册.txt', 'file_name': '问答手册.txt', 'file_type': 'text/plain', 'file_size': 3355, 'creation_date': '2025-03-20', 'last_modified_date': '2025-03-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ba78daa1-3743-415b-9aec-22f08e8029bf', node_type='4', metadata={'file_path': 'docs/问答手册.txt', 'file_name': '问答手册.txt', 'file_type': 'text/plain', 'file_size': 3355, 'creation_date': '2025-03-20', 'last_modified_date': '2025-03-20'}, hash='11aa24379c93cca29d0797f16e428d606adf5ecd542e488a07a29e676e324306')}, metadata_template='{key}: {value}', metadata_separator='\n', text='Mult

查看index下面的所有node的id

In [3]:
print(index.index_struct.nodes_dict)


{'cdf10de3-c206-4796-b6b2-3720e6f342dd': 'cdf10de3-c206-4796-b6b2-3720e6f342dd'}


查看index下面所有有ref的文档的信息


In [4]:
print(index.ref_doc_info)


{'ba78daa1-3743-415b-9aec-22f08e8029bf': RefDocInfo(node_ids=['cdf10de3-c206-4796-b6b2-3720e6f342dd'], metadata={'file_path': 'docs/问答手册.txt', 'file_name': '问答手册.txt', 'file_type': 'text/plain', 'file_size': 3355, 'creation_date': '2025-03-20', 'last_modified_date': '2025-03-20'})}


查看任意给定id的node详细信息

In [7]:
index.docstore.get_node('cdf10de3-c206-4796-b6b2-3720e6f342dd') # 需传入上一步的id
# 或者 index.docstore.docs['cdf10de3-c206-4796-b6b2-3720e6f342dd']

TextNode(id_='cdf10de3-c206-4796-b6b2-3720e6f342dd', embedding=None, metadata={'file_path': 'docs/问答手册.txt', 'file_name': '问答手册.txt', 'file_type': 'text/plain', 'file_size': 3355, 'creation_date': '2025-03-20', 'last_modified_date': '2025-03-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ba78daa1-3743-415b-9aec-22f08e8029bf', node_type='4', metadata={'file_path': 'docs/问答手册.txt', 'file_name': '问答手册.txt', 'file_type': 'text/plain', 'file_size': 3355, 'creation_date': '2025-03-20', 'last_modified_date': '2025-03-20'}, hash='11aa24379c93cca29d0797f16e428d606adf5ecd542e488a07a29e676e324306')}, metadata_template='{key}: {value}', metadata_separator='\n', text='Multimodal Agent AI systems have many applica

删除一个节点，删除这个操作尽量不要尝试，可能会导致后面的代码运行出错。


In [ ]:
# index.docstore.delete_document('51595901-ebe3-48b5-b57b-dc8794ef4556')

新增节点

In [9]:
# index.insert_nodes([doc_single])


注意这里的doc_single必须是一个 TextNode 对象。例如上文查看node时输出的那个。
TextNode 对象也可以自己构造。构造方式为：


In [10]:
from llama_index.core.schema import TextNode
nodes = [
    TextNode(
        text="The Shawshank Redemption",
        metadata={
            "author": "Stephen King",
            "theme": "Friendship",
            "year": 1994,
        },
    ),
    TextNode(
        text="The Godfather",
        metadata={
            "director": "Francis Ford Coppola",
            "theme": "Mafia",
            "year": 1972,
        },
    )
]
index.insert_nodes(nodes)

或者仿照前一节课的从文档构造节点的方式。


In [13]:
# 从指定文件读取，输入为List
from llama_index.core import SimpleDirectoryReader,Document
documents = SimpleDirectoryReader(input_files=['./docs/另一个手册.txt']).load_data()
from llama_index.core.node_parser import SentenceSplitter
transformations = [SentenceSplitter(chunk_size = 512)]

from llama_index.core.ingestion.pipeline import run_transformations
nodes = run_transformations(documents, transformations=transformations)
index.insert_nodes(nodes)
print(nodes)

[TextNode(id_='4e5de192-cf65-4356-b53a-9b82edf23947', embedding=None, metadata={'file_path': 'docs/另一个手册.txt', 'file_name': '另一个手册.txt', 'file_type': 'text/plain', 'file_size': 3355, 'creation_date': '2025-03-20', 'last_modified_date': '2025-03-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='fed2e427-2d65-45a2-99c0-d7c6ab22408a', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'docs/另一个手册.txt', 'file_name': '另一个手册.txt', 'file_type': 'text/plain', 'file_size': 3355, 'creation_date': '2025-03-20', 'last_modified_date': '2025-03-20'}, hash='11aa24379c93cca29d0797f16e428d606adf5ecd542e488a07a29e676e324306'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='76ed47a8-bb7c-4f17-b804-4f6b66

至于改的话，没有很方便的方式。先删除再新增吧。
